In [55]:
import os
import requests
from dotenv import load_dotenv
from mistralai import Mistral
import json

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

In [5]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
)

In [6]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: The Spine of the Leviathans

World Description: In the world of The Spine of the Leviathans, cities are built on the backs of colossal, sentient beasts called Leviathans. These creatures roam the endless plains, their massive forms supporting towering structures where humans live and thrive. The bond between the Leviathans and their human inhabitants is symbiotic, with each relying on the other for survival. The constant movement of the Leviathans means that the cities are always on the move, creating a dynamic and ever-changing landscape.


In [7]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [8]:
kingdoms_output = output.choices[0].message.content
print(kingdoms_output)

Kingdom 1 Name: Aerium
Kingdom 1 Description: Aerium is a kingdom of scholars and inventors, known for its advanced technology and magical innovations. The Leviathan, Skywhisper, is revered for its gentle nature and ability to reach great heights, allowing Aerium to trade with other kingdoms from above. The current ruler, Archon Lyra, is a brilliant engineer who has developed ways to harness the Leviathan's natural magic for the benefit of her people. Aerium's history is marked by a great schism, when a group of mages attempted to sever their Leviathan's bond, leading to a catastrophic event that reshaped the kingdom's approach to magic.

Kingdom 2 Name: Ignis
Kingdom 2 Description: Ignis is a fiery kingdom, built on the back of the Leviathan, Emberstride, which possesses the ability to breathe flames. The people of Ignis are skilled blacksmiths and warriors, forging weapons and armor of unparalleled quality. Queen Phoenix leads with an iron fist, her rule marked by constant expansion 

In [9]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aerium" in The Spine of the Leviathans
Created kingdom "Ignis" in The Spine of the Leviathans
Created kingdom "Verdantia" in The Spine of the Leviathans

Kingdom 1 Description: Verdantia is a lush, green kingdom, home to the Leviathan, Greenheart, which possesses the power to heal and grow plants. The people of Verdantia are skilled farmers and healers, living in harmony with nature. King Thalion, a wise and just ruler, governs with the counsel of the ancient treants that dwell within Greenheart's forests. Verdantia's history is one of peace and growth, as the kingdom has always sought to coexist with the other Leviathans and their human inhabitants.


In [10]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [11]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aerium...
Lumina
Lumina is nestled in a lush, verdant valley, surrounded by towering, bioluminescent crystals that cast an ethereal glow over the town. The Crystal Grotto, a vast network of caves filled with precious gems, is the town's most important place, attracting miners and adventurers. Lumina's history is steeped in legend, as it is said to have been founded by a group of light-wielding mages who were exiled from Aerium during the great schism.


Ironhold
Ironhold is a fortified town perched on the edge of the Whispering Expanse, a vast, windswept plain where the Leviathans' movements create powerful gusts. The Windforge, a massive foundry powered by wind energy, is the town's heart, producing weapons and armor for Aerium's armies. Ironhold was once a simple outpost, but it grew in importance after a legendary blacksmith discovered a way to infuse metal with the Leviathans' magic, making it nearly indestructible.


Serenity
Serenity is a tranquil tow

In [13]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    

In [29]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for num in range(1, 4):
        print(npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0])
        print(npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0])
        print("\n")

        character_name = npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0]
        character_description = npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0]

        character = {
        "name": character_name,
        "description": character_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[character_name] = character
        town["npcs"] = npcs

    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom and town
    break


Creating characters for the town of: Lumina...
Eamon
Eamon is a rugged, weather-beaten miner with calloused hands and a strong, sturdy build, his eyes gleam with the reflected light of the crystals he spends his days extracting. He is a skilled miner, known for his ability to navigate the treacherous Crystal Grotto. Deep down, Eamon is haunted by the loss of his wife in a mining accident, driving him to push the boundaries of safety in his quest to uncover the legendary Heart of Lumina, a gem said to grant immense power, hoping to use it to protect his remaining family.


Elara
Elara is a young, agile acrobat with a lithe frame and graceful movements, her eyes hold a spark of mischief and her hair is often adorned with small, glowing crystals. She performs in the town's theater, using her light-manipulating magic to create dazzling displays. Despite her public persona, Elara struggles with the stigma of being a mage in a town that fears and mistrusts magic, yearning to prove that her 

In [45]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Serenity, Aerium:
Elara is a young woman with long, silver hair and eyes that shimmer like the Silver River. She is a skilled healer, using her knowledge of herbs and the Leviathans' magic to aid the sick and injured. Elara's deep pain comes from the loss of her mentor, who was killed during a magical experiment gone wrong, fueled by the remnants of the great schism. She desires to understand the true nature of the Leviathans' magic to prevent such tragedies and to honor her mentor's legacy.


In [57]:
world["name"]

'The Spine of the Leviathans'

In [59]:
def save_world(world, filename):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Assuming 'world' is a dictionary with a 'name' key
#example world

save_world(world, f'../shared_data/{world["name"]}.json')